## Ray Data operation examples

### Configuration

In [13]:
! pip install --user -q "google-cloud-aiplatform[ray]>=1.56.0" \
                        "ray[data,train,tune,serve]>=2.33.0"

In [20]:
# @title Define constants
PROJECT_NBR = "721521243942"
PROJECT_ID = "ai-hangsik"
REGION="us-central1"
RAY_CLUSTER_NM = "ray33-cluster-20250216-192557"

In [21]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [22]:
import ray
from ray.runtime_env import RuntimeEnv
from ray.air.config import RunConfig
from ray.air import CheckpointConfig, ScalingConfig
from ray.util.joblib import register_ray

In [23]:
ray.__version__

'2.33.0'

In [62]:

ray.shutdown()

In [25]:

RAY_ADDRESS=f"vertex_ray://projects/{PROJECT_NBR}/locations/{REGION}/persistentResources/{RAY_CLUSTER_NM}"
print(f"RAY_ADDRESS:{RAY_ADDRESS}")

RUNTIME_ENV = {
  "pip": [
      "google-cloud-aiplatform[ray]>=1.56.0",
      "ray[data,train,tune,serve]>=2.33.0",
      "datasets",
      "evaluate",
      "accelerate==0.18.0",
      "transformers==4.26.0",
      "torch>=1.12.0",
      "deepspeed==0.12.3",
  ],
}

ray.init(address=RAY_ADDRESS,runtime_env=RUNTIME_ENV)

RAY_ADDRESS:vertex_ray://projects/721521243942/locations/us-central1/persistentResources/ray33-cluster-20250216-192557
[Ray on Vertex AI]: Cluster State = State.RUNNING


SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.16
Ray version:,2.33.0
Vertex SDK version:,1.80.0
Dashboard:,e8b73f504c907557-dot-us-central1.aiplatform-training.googleusercontent.com
Interactive Terminal Uri:,fe7820d66f7ca7e2-dot-us-central1.aiplatform-training.googleusercontent.com
Cluster Name:,ray33-cluster-20250216-192557


(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]
(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]
(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]


(load_data pid=2171, ip=10.127.0.100) Column         Type
(load_data pid=2171, ip=10.127.0.100) ------         ----
(load_data pid=2171, ip=10.127.0.100) Id             int64
(load_data pid=2171, ip=10.127.0.100) SepalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) SepalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) PetalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) PetalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) Species        string


(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]
(load_data pid=2171, ip=10.127.0.100) Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> LimitOperator[limit=1]


(load_data pid=2171, ip=10.127.0.100) Column         Type
(load_data pid=2171, ip=10.127.0.100) ------         ----
(load_data pid=2171, ip=10.127.0.100) Id             int64
(load_data pid=2171, ip=10.127.0.100) SepalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) SepalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) PetalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) PetalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) Species        string
(load_data pid=2171, ip=10.127.0.100) [{'Id': 1, 'SepalLengthCm': 5.1, 'SepalWidthCm': 3.5, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa'}]


(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]
(load_data pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(load_data pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> LimitOperator[limit=1]


(load_data pid=2171, ip=10.127.0.100) Column         Type
(load_data pid=2171, ip=10.127.0.100) ------         ----
(load_data pid=2171, ip=10.127.0.100) Id             int64
(load_data pid=2171, ip=10.127.0.100) SepalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) SepalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) PetalLengthCm  double
(load_data pid=2171, ip=10.127.0.100) PetalWidthCm   double
(load_data pid=2171, ip=10.127.0.100) Species        string
(load_data pid=2171, ip=10.127.0.100) [{'Id': 1, 'SepalLengthCm': 5.1, 'SepalWidthCm': 3.5, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa'}]


(transform_request pid=2171, ip=10.127.0.100) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data
(transform_request pid=2171, ip=10.127.0.100) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[MapBatches(RemoteFunction)]
(transform_request pid=2171, ip=10.127.0.100) An exception was raised from a task of operator "MapBatches(RemoteFunction)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.
(transform_request pid=2171, ip=10.127.0.100) Exception occurred in user code, with the abbreviated stack trace below. By default, the Ray Data internal stack trace is omitted from stdout, and only written to the Ray Data log files at /tmp/ray/session_2025-02-16_10-36-39_777647_11/logs/ray-data. To output the full stack trace to stdout, set `DataContext.log_internal_stack_trace_to_stdout` to True.
(save_data pid=2171, ip=10.127.0

### Basic Operation

#### Get data from GCS

* https://docs.ray.io/en/latest/data/api/doc/ray.data.read_csv.html#ray.data.read_csv

In [48]:

@ray.remote
def load_data():
    
    import ray
    
    ds = ray.data.read_csv("gs://sllm_checkpoints/data/Iris.csv" )
    print(ds.schema())
    print(ds.take(1))

In [49]:
ray.get(load_data.remote())

#### Different parse option

In [ ]:

@ray.remote
def load_data():
    
    from pyarrow import csv
    
    parse_options = csv.ParseOptions(delimiter="\t")
    ds = ray.data.read_csv("gs://sllm_checkpoints/data/Iris.tsv", parse_options=parse_options )
    print(ds.schema())


In [ ]:
ray.get(load_data.remote())

In [59]:
# https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.write_csv.html#ray.data.Dataset.write_csv

@ray.remote
def save_data():

    import os
    
    ds = ray.data.read_csv("gs://sllm_checkpoints/data/Iris.csv" )
    ds.write_csv("gs://sllm_checkpoints/tmp_store", num_rows_per_file = 1000)


In [60]:
ray.get(save_data.remote())